In [1]:
import os
import json
import shutil
import scrapbook as sb
# from ltp import LTP

# ltp = LTP()  # 默认加载 Small 模型

/opt/conda/lib/python3.7/site-packages/papermill/iorw.py:50: FutureWarning: pyarrow.HadoopFileSystem is deprecated as of 2.0.0, please use pyarrow.fs.HadoopFileSystem instead.
  from pyarrow import HadoopFileSystem


# Settings

In [2]:
src_dir = "../../nlp_pipeline"  # Ignore this!

# Pipeline settings
task = "sequence_classification"  # chinese_word_segmentation OR target_classification, OR sequence_classification
model = 'BERT_CLS'  # None: Default model 
device = 0
text_prepro = None  # None: Default steps 

model_params = {
    'num_train_epochs': 5, 
    'max_length': 256,
    'tokenizer_name': "bert-base-cased" ,
    'pretrained_lm': "bert-base-cased" ,
#     'embedding_trainable': True, 
#     'output_hidden_act_func': "PReLU", 
#     'output_hidden_dim': 128, 
#     'output_use_bn': False, 
#     'optimizer': "AdamW",
#     'learning_rate': 2e-5,
#     'weight_decay': 0.0,
#     'gradient_accumulation_steps': 1,
#     'adam_epsilon': 1e-8,
#     'max_grad_norm': 1.0,
}

train_params = {
    'batch_size': 16, 
    'seed': 42, 
    'optimization_metric': "macro_f1", 
    'early_stop': None
}

eval_params = {
    'batch_size': 32
}

# model_dir = f"../output/test_pipeline_{task}_sst"  # output dir for new model
# model_dir = f"../output/explainable_ai_paper/bert_cls_sst"
# model_dir = f"/ailab/shared/Users/quincy/canton-target-sentiment/output/explainable_ai_paper/bert_cls_sst"
model_dir = f"../output/knowledge_distillation/teacher_model_cls2"

In [3]:
os.chdir(src_dir)
from nlp_pipeline.pipeline import Pipeline

# Load data

In [4]:

# train_raw_data = json.load(open(f"../data/datasets/public/{task}/sst/train.json", 'r'))
# dev_raw_data = json.load(open(f"../data/datasets/public/{task}/sst/test.json", 'r'))
# test_raw_data = json.load(open(f"../data/datasets/public/{task}/sst/test.json", 'r'))
# print(train_raw_data[0])

In [5]:

train_raw_data = json.load(open(f"../data/datasets/internal/sequence_classification/post_sentiment/train.json", 'r'))
dev_raw_data = json.load(open(f"../data/datasets/internal/sequence_classification/post_sentiment/test.json", 'r'))
test_raw_data = json.load(open(f"../data/datasets/internal/sequence_classification/post_sentiment/test.json", 'r'))
print(train_raw_data[0])

{'content': '德國政府證實,在大衆旗下豪華汽車品牌奧迪的多款柴油車上發現專門應付尾氣排放檢測的作弊軟件,包括奧迪a8。『大衆旗下奧迪a8首曝"排放門"_手機新浪網』http://t.cn/rsatfqj', 'label': -1}


# Run pipeline

## Initialize pipeline

In [6]:

pipeline = Pipeline(
    task=task, 
    model=model, 
    device=device, 
    text_prepro=text_prepro
)

2021-12-07 04:01:40 ***** Model class is specified for sequence_classification. *****
2021-12-07 04:01:40   Model = BERT_CLS


../config/examples/sequence_classification/BERT_CLS
['.ipynb_checkpoints', 'run.yaml', 'model', 'result', 'logs']


## Train a new model

In [ ]:

pipeline.train(
    model_dir, 
    train_raw_data=train_raw_data, 
    dev_raw_data=dev_raw_data, 
    model_params=model_params,
    train_params=train_params
)


2021-12-07 04:01:40 ***** Initializing pipeline *****
2021-12-07 04:01:40 ***** Loading tokenizer *****
2021-12-07 04:01:40   Tokenizer source = 'transformers'
2021-12-07 04:01:40 ***** Initializing model *****
2021-12-07 04:01:40   Task = sequence_classification
2021-12-07 04:01:40   Model class = BERT_CLS
2021-12-07 04:01:40   Model path = ../output/knowledge_distillation/teacher_model_cls2/model/model.pt


['run.yaml', 'model.yaml', 'tokenizer', 'label_to_id.json', 'model.pt']


2021-12-07 04:01:48 ***** Loading data *****
2021-12-07 04:01:48   Raw data is provided.
86637it [10:29, 166.14it/s]

## Test

In [13]:

metrics = pipeline.test(
    test_raw_data=test_raw_data,
    eval_params=eval_params
)


2021-11-30 02:10:28 ***** Loading data *****
2021-11-30 02:10:28   Raw data is provided.
1821it [00:07, 233.53it/s]
2021-11-30 02:10:35   Loaded samples = 1821
2021-11-30 02:10:35 ***** Running evaluation *****
2021-11-30 02:10:35   Num examples = 6920
2021-11-30 02:10:35   Batch size = 32
Evaluating: 100%|██████████| 217/217 [00:36<00:00,  5.96it/s]
2021-11-30 02:11:12   accuracy = 0.9992774566473989
2021-11-30 02:11:12   macro_f1 = 0.9992761232323626
2021-11-30 02:11:12   micro_f1 = 0.9992774701162376
2021-11-30 02:11:12   support = 6920
2021-11-30 02:11:12   0-precision = 0.9987926350739511
2021-11-30 02:11:12   0-recall = 0.9996978851963746
2021-11-30 02:11:12   0-f1-score = 0.9992450551109768
2021-11-30 02:11:12   0-support = 3310
2021-11-30 02:11:12   1-precision = 0.9997227612974772
2021-11-30 02:11:12   1-recall = 0.9988919667590028
2021-11-30 02:11:12   1-f1-score = 0.9993071913537482
2021-11-30 02:11:12   1-support = 3610
2021-11-30 02:11:12   loss = 0.003224152470788648
2021

## Load and Predict

In [6]:

pipeline = Pipeline(
    model_dir=model_dir, 
    device=device, 
)

print("Input:")
print(test_raw_data[0])

output = pipeline.predict(
    data_dict=test_raw_data[0],
)

print("Output:")
print(output)

2021-12-07 02:36:44 ***** Existing model is provided. *****
2021-12-07 02:36:44   Model directory = ../output/knowledge_distillation/teacher_model_cls
2021-12-07 02:36:44 ***** Initializing pipeline *****
2021-12-07 02:36:44 ***** Loading tokenizer *****
2021-12-07 02:36:44   Tokenizer source = 'transformers'
2021-12-07 02:36:44 ***** Initializing model *****
2021-12-07 02:36:44   Task = sequence_classification
2021-12-07 02:36:44   Model class = BERT_CLS
2021-12-07 02:36:44   Model path = ../output/knowledge_distillation/teacher_model_cls/model/model.pt


../output/knowledge_distillation/teacher_model_cls/model
['tokenizer', 'label_to_id.json', 'run.yaml', 'model.yaml', 'model.pt']
['tokenizer', 'label_to_id.json', 'run.yaml', 'model.yaml', 'model.pt']
Input:
{'content': '# 现在的选手很少有这种强悍气势啊', 'label': '1'}
Output:
{'prediction_id': 1, 'prediction': '1'}


### VisualizationDataRecord

In [1]:
from captum.attr import visualization as viz
import numpy as np

In [8]:

# raw_data = test_raw_data[14]
# visualize_data_record_bert(pipeline, raw_data)

In [4]:
?viz.visualize_text

Signature:
viz.visualize_text(
    datarecords: Iterable[captum.attr._utils.visualization.VisualizationDataRecord],
    legend: bool = True,
) -> 'HTML'
Docstring: <no docstring>
File:      /opt/conda/lib/python3.7/site-packages/captum/attr/_utils/visualization.py
Type:      function


In [9]:
import requests


def get_word_segments(raw_text):

    data = requests.post("https://ai.wws.wisers.net/ltp-commercial/ltp/article/single/pos", json={'content': raw_text}).json()
    segments = []
    segments_idxs = []
    for sent_res in data['content']['sent_res']:
        segments += sent_res['words']
        segments_idxs += sent_res['art_indexs']
        
    return segments, segments_idxs

In [10]:

# seg, hidden = ltp.seg(["他叫汤姆去拿外衣。"])


def get_explanation(pipeline, raw_data, enable_faithfulness=False):
    tokens, scores, attr_target, attr_target_prob, tokens_encoded, comprehensiveness = pipeline.explain(
        data_dict=raw_data,
        method='IntegratedGradients',
        layer='pretrained_model.embeddings.word_embeddings', 
        norm='sum',
        enable_faithfulness=enable_faithfulness  # use comp if True
    )
    return tokens, scores, attr_target, attr_target_prob, tokens_encoded, comprehensiveness
    

def visualize_data_record_bert(pipeline, raw_data, tokens, scores, attr_target, attr_target_prob):
    
    true_class = raw_data['label']
    attr_class = pipeline.args.label_to_id_inv[attr_target]
    start_position_vis = viz.VisualizationDataRecord(
                            scores,
                            pred_prob=attr_target_prob,
                            pred_class=attr_class,
                            true_class=true_class,
                            attr_class=attr_class,
                            attr_score=np.sum(scores),       
                            raw_input=tokens,
                            convergence_score=None)
    viz.visualize_text([start_position_vis])
    
    
def get_max_magnitude(arr):
    return arr[np.argmax(np.absolute(arr))]

    
def get_segment_level_explanation(seg_func, raw_text, tokens, scores, tokens_encoded):
    """
    Args:
        scores: token level scores
    """
    
    # to char level scores
    scores_char = []
    for char_id in range(len(raw_text)):
        token_id = tokens_encoded.char_to_token(char_id)
        if token_id is not None:
            scores_char.append((token_id, scores[token_id]))
        else:
            scores_char.append((token_id, 0))
    
    # get segments indexs
    segments, segments_idxs = seg_func(raw_text)
    
    # seg level scores
    scores_seg = []
    cur_idx = 0
    for idx in segments_idxs:
        scores_ = scores_char[idx[0]:idx[1]]
        scores_ = [x[-1] for x in list(set(scores_))]
        scores_seg.append(
            np.sum(scores_)
        )
    
    return segments, scores_seg
        
    

In [11]:
for i, raw_data in enumerate(test_raw_data[:50]):
    print(i, "#########################")
    tokens, scores, attr_target, attr_target_prob, tokens_encoded, faithfulness = get_explanation(
        pipeline, 
        raw_data, 
        enable_faithfulness=True
    )
    
    print("Faithfulness (COMP) =", faithfulness)
    visualize_data_record_bert(
        pipeline, 
        raw_data, 
        tokens=tokens, 
        scores=scores, 
        attr_target=attr_target, 
        attr_target_prob=attr_target_prob
    )

    tokens_seg, scores_seg = get_segment_level_explanation(
        seg_func=get_word_segments, 
        raw_text=raw_data['content'], 
        tokens=tokens, 
        scores=scores, 
        tokens_encoded=tokens_encoded
    )

    visualize_data_record_bert(
        pipeline, 
        raw_data, 
        tokens=tokens_seg, 
        scores=scores_seg, 
        attr_target=attr_target, 
        attr_target_prob=attr_target_prob
    )

0 #########################
Faithfulness (COMP) = 0.2756514136058589


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
1,1 (0.45),1,1.87,[CLS] # 现 在 的 选 手 很 少 有 这 种 强 悍 气 势 啊 [SEP]


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
1,1 (0.45),1,1.87,# 现在 的 选手 很少 有 这种 强悍 气势 啊


1 #########################
Faithfulness (COMP) = 0.825786868826701


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
0,1 (0.94),1,1.46,三钢闽光 最佳 买 、 卖 点位 得出 ， 战友们 不要 被 主力 阴谋 所 迷惑


...

45 #########################
Faithfulness (COMP) = 0.7403398538854989


46 #########################
Faithfulness (COMP) = 0.617358037693934


47 #########################
Faithfulness (COMP) = 0.594339518994093


48 #########################
Faithfulness (COMP) = 0.319339660461992


49 #########################
Faithfulness (COMP) = 0.1679503253915093


# Clear output folder

In [ ]:
shutil.rmtree(model_dir)

# Export variables

In [10]:
sb.glue("macro_f1", metrics['macro_f1'])
sb.glue("micro_f1", metrics['micro_f1'])